<a href="https://colab.research.google.com/github/arasmasood/aras/blob/master/Copy_of_Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import struct
import socket
import time
import datetime
from datetime import datetime, timedelta
import warnings
import tkinter as tk  # importing Tkinter library
from collections.abc import Iterable
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from numpy import mean
from numpy import std
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
# import matplotlib.pylab as plt
from sklearn.model_selection import cross_val_score
from time import perf_counter, sleep
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from tkinter import *
from tkinter.ttk import *
from sklearn import preprocessing
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import lightgbm as lgb
global df_rows
global df
global file_path
global data
global label_Anomaly_data
global label_Normal_data
from BorutaShap import BorutaShap, load_data
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
import shap
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# copy
class MyApp:
    def __init__(self):
        
       
        self.df = pd.read_csv("C:\\Users\\micro\\IOTDATA\\IOTTT.csv")
        self.pre_Process(self.df)
        cleaned_data = self.feature_scaling(self.df)
        
        #print("The execution time for preprocssing and feature scaling is: ")
        #self.elapced_time(self.start,time.time())
        
        #self.feature_selection_RandomForest(cleaned_data)
        self.feature_selection(cleaned_data)
        #self.feature_selection_XGBoost(cleaned_data)
        #self.feature_selection_GBoost(cleaned_data)
    def feature_selection_GBoost(self,df):
        print("\n**** Feature Selection Starting GBoost****\n")
        
        X = df.drop("Label",axis=1)
        y = df["Label"]
        
        numNormal = len(self.df[self.df["Label"]==1.0])
        numAnomaly = len(self.df[self.df["Label"]==0.0])
        
        SPW = numAnomaly / numNormal
        
        model = GradientBoostingClassifier(n_estimators=10)
        Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)
        start = time.time()
        Feature_Selector.fit(X=X, y=y, n_trials=10, sample=False,
                              normalize=True,
                     verbose=True)
        end = time.time()
        Feature_Selector.plot(which_features='all')
        
        subset = Feature_Selector.Subset()

        print(subset)
        print("\n ***** Feature Selection End *****\n")
        
        print("The execution time for GBoost Algorithm is: ")
        print(end-start)
        
    def feature_selection_XGBoost(self, df):
        #---------------splitting-----------------------------
        print("\n ***** Feature Selection Startin XGB *****\n")
        X = df.drop('Label', axis=1)
        y = df['Label']
        numNormal = len(self.df[self.df['Label'] == 1.0])
        numAnomaly = len(self.df[self.df['Label'] == 0.0])

       
  
        SPW = numAnomaly / numNormal
        
        model = XGBClassifier( scale_pos_weight=SPW, objective='binary:logistic', random_state=1,tree_method='gpu_hist', gpu_id=0)
        Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)
        start = time.time()
        Feature_Selector.fit(X=X, y=y, n_trials=50, sample=False,
                              normalize=True,
                     verbose=True)
        end = time.time()
        Feature_Selector.plot(which_features='all')
        
        subset = Feature_Selector.Subset()

        print(subset)
        print("\n ***** Feature Selection End *****\n")
        
        print("The execution time for XGBoost Algorithm is: ")
        print(end-start)
        
    def feature_selection_RandomForest(self, df):
        #---------------splitting-----------------------------
        print("\n ***** Feature Selection Startin RandomForest *****\n")
        X = df.drop('Label', axis=1)
        y = df['Label']
    
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)
        
        model = BalancedRandomForestClassifier()
        Feature_Selector = BorutaShap(model=model,
                        importance_measure='shap',
                              classification=True)
        start = time.time()
        Feature_Selector.fit(X_train, y_train,n_trials=50)
        end = time.time()
        
        #sel.get_support()
        #selected_feat = X_train.columns[(sel.get_support())]
        #len(selected_feat)
        selected_columns = list()
        selected_columns.append(sorted(Feature_Selector.Subset().columns))
        print(len(selected_columns))
        print("Important column names: ")
        print(selected_columns)
        
        print("\n ***** Feature Selection End *****\n")
        
        print("The execution time for RandomForest Algorithm is: ")
        print(end-start)
    
    def feature_selection(self, df):
        #---------------splitting-----------------------------
        X = df.drop('Label', axis=1)
        y = df['Label']
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)       
        #-------------------------------------------       
        print("\n ***** Feature Selection Starts *****\n")
        
        
        print("\n X Section \n")
        print(X)
        
        print("\n Y Section \n")
        print(y)
        
        model = RandomForestClassifier()
        Feature_Selector = BorutaShap(model=model,importance_measure='shap',
                                  classification=True)
        start = time.time()
        Feature_Selector.fit(X=X_train, y=y_train, n_trials=50, sample=False,
                            normalize=True,
                             verbose=True)
        end = time.time()
        Feature_Selector.plot(which_features='all')
        subset = Feature_Selector.Subset()

        print(subset)
        
        print("\n ***** Feature Selection End *****\n")
        
        print("The execution time for feauter selection is: ")
        print(end-start)


    def normal_And_Anom_lable(self, excel_filename):
        global dataframe
        dataframe = pd.read_csv(excel_filename)
        Number_of_Rows = len(self.df)

        Normal = dataframe["Label"].sum().count('Normal')
        Anomaly = dataframe["Label"].sum().count('Anomaly')

        self.df.Label.apply(lambda x: 0 if x == 'Normal' else 1)
        Lable_for_Normal_Data = 'Number of Normals     : ' + str(Normal) + " {Normals are  = 0 }"
        Lable_for_Anomal_Data = 'Number of Anomalys    : ' + str(Anomaly) + " {Anomalys are = 1 }"

        LND = Lable_for_Normal_Data
        Anomal_Datas = Lable_for_Anomal_Data
        label_Anomaly_data["text"] = Anomal_Datas
        label_Normal_data["text"] = LND


    def pre_Process(self, df):
        print("\n ***** Pre Processing Starts ***** \n")

        str = self.isInf_isNan(self.df)

        str1 = self.Duplicaete(self.df)

        str2 = self.drop_Dataframe(self.df)

        str3 = self.Convert_Ip(self.df)

        str4 = self.label_Creator(self.df)

        strTotal = str + str1 + str2 + str3 + str4

        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]

        label_encoder = preprocessing.LabelEncoder()

        # Encode labels in column 'species'.
        self.df['Label'] = label_encoder.fit_transform(self.df['Label'])

        self.df['Label'].unique()

        print("\n", strTotal, "\n")

        print(self.df)

        print("\n ***** Pre Processing Ends ***** \n")


    def feature_scaling(self, df):
        print("\n ***** Feature Scaling Starts ***** \n")
        # label_tmp = self.df["Label"]
        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]


        columnsNames = list(self.df.columns)

        # feature scaling code
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)

        df_X = pd.DataFrame(X, columns=columnsNames[:-1])
        df_X.reset_index(drop=True, inplace=True)
        df_Y = pd.DataFrame(y)
        df_Y.reset_index(drop=True, inplace=True)

        self.df = pd.concat([df_X, df_Y] ,axis=1)



        print(self.df)
        #self.df=self.df[0:1000]
        #self.df.to_csv("C:\\Users\\micro\\IOTDATA\\Output.csv")
    
        return self.df
        print("\n ***** Feature Scaling Ends ***** \n")


    def isInf_isNan(self, df):
        # Removing NULL and INF Code Starts
        is_nan = self.df.isin([np.nan]).any(axis=1)
        is_inf = self.df.isin([np.inf, -np.inf]).any(axis=1)
        tmp1 = str(self.df[is_inf].shape[0])
        tmp2 = str(self.df[is_nan].shape[0])
        prev_count = self.df.shape[0]
        self.df = self.df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
        aft_count = self.df.shape[0]
        isNanisInfResult = "INF Found: " + tmp1 + " || NaN Found: " + tmp2 + "\n"
        isNanisInfResult += "Removed {} Now {}".format(prev_count - aft_count, self.df.shape) + " "

        return isNanisInfResult

    def Duplicaete(self, df):
        # Removing Duplicates Code Starts
        sum = self.df.duplicated().sum()
        # df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
        self.df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

        dupResult = "\n-------------------------------\n"
        dupResult += str(sum) + " are duplicates and removed"
        return dupResult

    # Removing Duplicates Code Ends
    def drop_Dataframe(self, df):
        self.df.drop(["Flow_ID", "Timestamp", "Cat", "Sub_Cat"], axis=1, inplace=True)
        dropDFResult = "\n-------------------------------\n"
        dropDFResult += "Flow_ID, Timestamp, Cat, Sub_Cat Columns are removed"

        return dropDFResult

    def ip2int(self, addr):
        # Changing SRC_IP, SDT_IP To Numeric Code Starts

        # changes the ip addresses to integer
        # print("* Changing SRC_IP, SDT_IP To Numeric *")
        return struct.unpack("!I", socket.inet_aton(addr))[0]

    def Convert_Ip(self, df):
        # Changing SRC_IP, SDT_IP To Numeric Code Ends
        # saving the resut inside the columns
        self.df['Src_IP'] = df.Src_IP.apply(lambda x: self.ip2int(x))
        self.df['Dst_IP'] = df.Dst_IP.apply(lambda x: self.ip2int(x))

        convertIPResult = "\n-------------------------------\n"
        convertIPResult += "Src_IP and  Dst_IP are converted to Int64 from Object"
        return convertIPResult

    def label_Creator(self, df):
        Type_of_data = self.df['Label']
        Type_of_data.replace({"Normal": "0", "Anomaly": "1", 'NUll': "1"}, inplace=True)

        Normal = self.df["Label"].sum().count('0')
        Anomaly = self.df["Label"].sum().count('1')
        labelCreator = "\n-------------------------------\n"
        labelCreator += "Normal -> " + str(Normal) + " || Anomaly ->" + str(Anomaly)

        return labelCreator

    # Models
    def XGBoostClassiferWithCV(self, df):

        print("\n *******************************")
        print("******XGBoost with CV*******")
        print("*******************************")
        numNormal = len(self.df[self.df['Label'] == 1.0])
        numAnomaly = len(self.df[self.df['Label'] == 0.0])

        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]
        print("The X values: ")
        print(X)

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)

        SPW = numAnomaly / numNormal
        xgb = XGBClassifier(scale_pos_weight=SPW, objective='binary:logistic', random_state=1, tree_method='gpu_hist', gpu_id=0)
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)



        cv_start = time.time()
        n_scores = cross_val_score(xgb, X, y, scoring='accuracy', cv=skf, n_jobs=-1, error_score='raise')
        cv_end = time.time()

        print('Cross validation score accuracy average for the model: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
        print("Accuracy values: ", np.round(n_scores, 4))
        print("\nTime consumed for Cross validation: %4.3f" % (cv_end - cv_start))

    def XGBoostClassiferWithOutCV(self, df):
        print("\n *******************************")
        print("******XGBoost without CV*******")
        print("*******************************")
        numNormal = len(self.df[self.df['Label'] == 1.0])
        numAnomaly = len(self.df[self.df['Label'] == 0.0])

        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]

        print(X)

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)

        SPW = numAnomaly / numNormal

        xgb = XGBClassifier(scale_pos_weight=SPW, objective='binary:logistic', random_state=1,tree_method='gpu_hist', gpu_id=0)

        training_start = time.time()

        xgb.fit(X_train, y_train)

        training_end = time.time()

        prediction_start = time.time()

        preds = xgb.predict(X_valid)

        prediction_end = time.time()

        preds_train = xgb.predict(X_train)

        accuracy_test = accuracy_score(y_valid, preds)  # compute test accuracy
        accuracy_tarin = accuracy_score(y_train, preds_train)  # compute train accuracy

        cm = confusion_matrix(y_valid, preds)

        print("XGBoost's testing accuracy is: %3.4f" % (accuracy_test * 100))
        print("XGBoost's training accuracy is: %3.4f" % (accuracy_tarin * 100))

        print("Precision:{}", precision_score(y_valid, preds))
        print("Recall:{}", recall_score(y_valid, preds))

        print("\nTime consumed for training the model %4.3f" % (training_end - training_start))
        print("Time consumed for testing: %6.5f seconds" % (prediction_end - prediction_start))

        print("Confusion Matrix: \n", cm)


    def LightGBMClassifer(self, df):
        print("\n *******************************")
        print("******LGBM with CV*******")
        print("*******************************")
        numNormal = len(self.df[self.df['Label'] == 1.0])
        numAnomaly = len(self.df[self.df['Label'] == 0.0])

        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)

        SPW = numAnomaly / numNormal

        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
        lgbm = LGBMClassifier(scale_pos_weight=SPW, random_state=1)

        cv_start = time.time()
        n_scores = cross_val_score(lgbm, X, y, scoring='accuracy', cv=skf, n_jobs=-1, error_score='raise')
        cv_end = time.time()

        print('Cross validation score accuracy average for the model: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
        print("Accuracy values: ", np.round(n_scores, 4))
        print("\nTime consumed for Cross validation: %4.3f" % (cv_end - cv_start))

    def LightGBMClassiferWithOutCV(self, df):
        print("\n *******************************")
        print("******LGBM without CV*******")
        print("*******************************")
        numNormal = len(self.df[self.df['Label'] == 1.0])
        numAnomaly = len(self.df[self.df['Label'] == 0.0])

        X = self.df.iloc[:, 0:-1]
        y = self.df.iloc[:, -1]

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)

        SPW = numAnomaly / numNormal

        lgbm = LGBMClassifier(scale_pos_weight=SPW, random_state=1)

        training_start = time.time()

        lgbm.fit(X_train, y_train)  # training phase

        training_end = time.time()

        prediction_start = time.time()

        preds = lgbm.predict(X_valid)  # test phase

        prediction_end = time.time()

        # prediction_train_start = time.time()

        preds_train = lgbm.predict(X_train)

        # prediction_train_end = time.time()

        accuracy_test = accuracy_score(y_valid, preds)
        accuracy_tarin = accuracy_score(y_train, preds_train)

        cm = confusion_matrix(y_valid, preds)

        print("LightBoost's testing accuracy is: %3.4f" % (accuracy_test * 100))
        print("LightBoost's training accuracy is: %3.4f" % (accuracy_tarin * 100))

        print("Precision:{}", precision_score(y_valid, preds))
        
        print("Recall:{}", recall_score(y_valid, preds))

        print("\nTime consumed for training the model %4.3f" % (training_end - training_start))
        print("Time consumed for testing: %6.5f seconds" % (prediction_end - prediction_start))


        print("Confusion Matrix: \n", cm)